In [50]:
# Flatten Harvard data
# For data manipulation
import os
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
PROJ_ROOT = os.pardir

# For Scraping
from bs4 import BeautifulSoup
import requests
import time

# For munging
import re
import json
import math

In [51]:
playerPath = os.path.join(PROJ_ROOT, "data", "interim", "harvard_players.json")

In [52]:
players = pd.read_json(playerPath)

In [53]:
players.head()

,B/T,City,Event,High School,Hometown,Ht.,Name,Name_link,No.,Position,Region,S/C,Weapon,Wt.,Yr.,season,sport
0,R/R,Long Beach,None,Woodrow Wilson,"Long Beach, Calif.",6-2,Tyler Albright,/sports/bsb/2008-09/bios/albright_tyler,6,C,Calif.,None,None,190,Jr.,2008-09,bsb
1,R/R,Austin,None,St. Stephens Episcopal,"Austin, Texas",5-10,Cole Arledge,/sports/bsb/2008-09/bios/arledge_cole,8,C,Texas,None,None,180,Jr.,2008-09,bsb
10,S/R,Winnetka,None,New Trier,"Winnetka, Ill.",6-3,Zach Hofeld,/sports/bsb/2008-09/bios/hofeld_zach,29,RHP,Ill.,None,None,205,Jr.,2008-09,bsb
100,L/R,Charlottesville,None,St. Anne's-Belfield,"Charlottesville, Va.",5-11,Jake Allen,/sports/bsb/2016-17/bios/allen_jake_gyf5,10,C,Va.,None,None,185,So.,2016-17,bsb
1000,None,Greendale,None,Greendale HS,"Greendale, Wis.",6-8,Eric Wanta,/sports/mbkb/1984-85/Bios/Eric_Wanta,15,Forward,Wis.,None,None,210,Freshman,1984-85,mbkb


In [54]:
players.shape

(10896, 17)

In [55]:
bio_parsed_path = os.path.join(PROJ_ROOT, "data", "interim", "harvard_bio_parsed.csv")
bios = pd.read_csv(bio_parsed_path)

In [56]:
bios.shape
bios.columns = ["Name_link", "synopsis", "headers"]

In [57]:
combined = players.merge(bios, on="Name_link", how="inner")

In [58]:
combined.shape

(10895, 19)

In [59]:
combined.head()

,B/T,City,Event,High School,Hometown,Ht.,Name,Name_link,No.,Position,Region,S/C,Weapon,Wt.,Yr.,season,sport,synopsis,headers
0,R/R,Long Beach,None,Woodrow Wilson,"Long Beach, Calif.",6-2,Tyler Albright,/sports/bsb/2008-09/bios/albright_tyler,6,C,Calif.,None,None,190,Jr.,2008-09,bsb,\r\nJunior (2010):\r\n\r\n Named team's captai...,"{""Position:"": ""C"", ""Year:"": ""Jr."", ""Height:"": ..."
1,R/R,Long Beach,None,Woodrow Wilson,"Long Beach, Calif.",6-2,Tyler Albright,/sports/bsb/2008-09/bios/albright_tyler,6,C,Calif.,None,None,190,Jr.,2009-10,bsb,\r\nJunior (2010):\r\n\r\n Named team's captai...,"{""Position:"": ""C"", ""Year:"": ""Jr."", ""Height:"": ..."
2,R/R,Austin,None,St. Stephens Episcopal,"Austin, Texas",5-10,Cole Arledge,/sports/bsb/2008-09/bios/arledge_cole,8,C,Texas,None,None,180,Jr.,2008-09,bsb,\r\nJunior (2010):\r\n\r\n Started at catcher ...,"{""Position:"": ""C"", ""Year:"": ""Jr."", ""Height:"": ..."
3,R/R,Austin,None,St. Stephens Episcopal,"Austin, Texas",5-10,Cole Arledge,/sports/bsb/2008-09/bios/arledge_cole,8,C,Texas,None,None,180,Jr.,2009-10,bsb,\r\nJunior (2010):\r\n\r\n Started at catcher ...,"{""Position:"": ""C"", ""Year:"": ""Jr."", ""Height:"": ..."
4,S/R,Winnetka,None,New Trier,"Winnetka, Ill.",6-3,Zach Hofeld,/sports/bsb/2008-09/bios/hofeld_zach,29,RHP,Ill.,None,None,205,Jr.,2008-09,bsb,\r\nSophomore (2009):\r\n\r\nStruck out three ...,"{""Position:"": ""RHP"", ""Year:"": ""Jr."", ""Height:""..."


In [60]:
def formatBio(row):
    synopsis = row["synopsis"]
    if type(synopsis) == float:
        return ""
    return re.sub("\n", "", synopsis)

In [61]:
combined["Bio"] = combined.apply(formatBio, axis=1)

In [62]:
del(combined["synopsis"])

In [63]:
playersSorted = combined.sort_values(["season"], ascending=False)

In [64]:
# Get range of years a player was involved in the sport
groupedFirst = playersSorted.groupby(by=["Name"]).first()
groupedLast = playersSorted.groupby(by=["Name"]).last()

In [65]:
len(groupedLast)

4270

In [66]:
for i in range(len(groupedLast)):
    first = groupedFirst.iloc[i]
    groupedLast.iloc[i]["season"] = groupedLast.iloc[i]["season"][:-2] + first["season"][-2:]

In [67]:
groupedLast.head()

,B/T,City,Event,High School,Hometown,Ht.,Name_link,No.,Position,Region,S/C,Weapon,Wt.,Yr.,season,sport,headers,Bio
Name,,,,,,,,,,,,,,,,,,
A.J. Jaffe,None,St. Charles,None,Marmion Academy,"St. Charles, Ill.",None,/sports/wrest/2016-17/bios/jaffe_aj,None,141,Ill.,None,None,None,Freshman,2016-17,wrest,"{""Hometown:"": ""St. Charles, Ill."", ""Position:""...",\rBefore Harvard Four-time letterwinner at Mar...
A.J. Kennedy,None,Fonthill,None,Notre Dame College,"Fonthill, Ont.",6-3,/sports/fball/2003-04/bios/a.j._kennedy,98,DE,Ont.,None,None,220,Freshman,2003-04,fball,"{""Position:"": ""DE"", ""Year:"": ""Freshman"", ""Heig...",
AJ Carvalho,None,Swansea,None,Joseph Case,"Swansea, Mass.",None,/sports/mcrew-lw/2010-11/bios/varsity/Carvalho...,None,None,Mass.,None,None,None,Sophomore,2010-13,mcrew-lw,"{""name"": ""Antonio Carvalho"", ""Year:"": ""Sophomo...",\rSophomore • 2011\r\rDid not see spring race ...
Aaki Vora,None,Mumbai,Freestyle,The Cathedral and John Connon School,"Mumbai, India",None,/sports/wswimdive/2016-17/bios/Vora_Aaki,None,None,India,None,None,None,Freshman,2016-17,wswimdive,"{""Hometown:"": ""Mumbai, India"", ""Position:"": ""F...",\rBefore Harvard\r\r Best female athlete at th...
Aaron Byrd,None,Abilene,None,Cooper,"Abilene, Texas",6-2,/sports/fball/2001-02/bios/aaron_byrd,3,DB,Texas,None,None,190,Freshman,2001-03,fball,"{""Position:"": ""DB"", ""Year:"": ""Freshman"", ""Heig...",


In [75]:
# Export data to csv
processedPath = os.path.join(PROJ_ROOT, "data", "interim", "harvard_bios_processed")
# groupedLast.to_csv(processedPath, encoding='utf-8')
groupedLast = groupedLast.reset_index()
groupedLast.to_pickle(processedPath + ".p")

In [77]:
groupedLast.head()

,Name,B/T,City,Event,High School,Hometown,Ht.,Name_link,No.,Position,Region,S/C,Weapon,Wt.,Yr.,season,sport,headers,Bio
0,A.J. Jaffe,None,St. Charles,None,Marmion Academy,"St. Charles, Ill.",None,/sports/wrest/2016-17/bios/jaffe_aj,None,141,Ill.,None,None,None,Freshman,2016-17,wrest,"{""Hometown:"": ""St. Charles, Ill."", ""Position:""...",\rBefore Harvard Four-time letterwinner at Mar...
1,A.J. Kennedy,None,Fonthill,None,Notre Dame College,"Fonthill, Ont.",6-3,/sports/fball/2003-04/bios/a.j._kennedy,98,DE,Ont.,None,None,220,Freshman,2003-04,fball,"{""Position:"": ""DE"", ""Year:"": ""Freshman"", ""Heig...",
2,AJ Carvalho,None,Swansea,None,Joseph Case,"Swansea, Mass.",None,/sports/mcrew-lw/2010-11/bios/varsity/Carvalho...,None,None,Mass.,None,None,None,Sophomore,2010-13,mcrew-lw,"{""name"": ""Antonio Carvalho"", ""Year:"": ""Sophomo...",\rSophomore • 2011\r\rDid not see spring race ...
3,Aaki Vora,None,Mumbai,Freestyle,The Cathedral and John Connon School,"Mumbai, India",None,/sports/wswimdive/2016-17/bios/Vora_Aaki,None,None,India,None,None,None,Freshman,2016-17,wswimdive,"{""Hometown:"": ""Mumbai, India"", ""Position:"": ""F...",\rBefore Harvard\r\r Best female athlete at th...
4,Aaron Byrd,None,Abilene,None,Cooper,"Abilene, Texas",6-2,/sports/fball/2001-02/bios/aaron_byrd,3,DB,Texas,None,None,190,Freshman,2001-03,fball,"{""Position:"": ""DB"", ""Year:"": ""Freshman"", ""Heig...",
